<head>
<b><center>CIENCIA DE DATOS</center>

<center>TALLER 3</center>
<center>2024-1</center>

<center>Profesor: Gabriel Jara </center><br>
<center>Integrantes: Fernando Salgado, Cristián González</center><br>
</head>

Intro

Carga Datos

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


exitosas = pd.read_csv('data/transacciones_exitosas.csv')
fallidas = pd.read_csv('data/transacciones_fallidas.csv')
evaluacion = pd.read_csv('data/transacciones_evaluación.csv')

# Eliminar fila con mal formato
exitosas = exitosas.drop(148541)


# Añadir etiquetas a los datos
exitosas['label'] = 0
fallidas['label'] = 1

# Combinar datos exitosos y fallidos
datos = pd.concat([exitosas, fallidas], ignore_index=True)



C:\Users\borrar\AppData\Local\Temp\ipykernel_5376\4292269185.py:11: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200) have mixed types. Specify dtype option on import or set low_memory=False.
  exitosas = pd.read_csv('data/transacciones_exitosas.csv')


Preprocesamiento

c:\Users\borrar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1140: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\borrar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1145: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\borrar\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\extmath.py:1165: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


Selección de Modelo

mejor modelo